In [2]:
import bcrypt
import psycopg2
# Connect to your postgres DB
conn = psycopg2.connect("dbname=appsenindb user=postgres password=admin host=localhost")

# Open a cursor to perform database operations
cur = conn.cursor()

In [3]:

def create_user(username, plain_password):
    # Generate salt
    salt = bcrypt.gensalt()
    
    # Hash password
    hashed_password = bcrypt.hashpw(plain_password.encode('utf-8'), salt)
    
    # Format data pengguna
    user_data = {
        "username": username,
        "password": hashed_password.decode('utf-8')  # Ubah byte ke string untuk disimpan di DB
    }
    
    return user_data

In [5]:
def decode_password(hashed_password, plain_password):
    # Decode hashed password
    if bcrypt.checkpw(plain_password.encode('utf-8'), hashed_password.encode('utf-8')):
        return True
    else:
        return False

In [14]:
# Contoh penggunaan
new_user = create_user("admin_appsenin", "secureAdmin#2025")
print(decode_password(new_user['password'], "secureAdmin#2025"))

print(new_user)

True
{'username': 'admin_appsenin', 'password': '$2b$12$et7XG3WVnQLD2MUbiID7jOtzDxiBg0Is6JdLtEWCnO7HC8S.noIAO'}


In [19]:
try:
	# Insert new user into at_sys_msuser table
	insert_query = """
	INSERT INTO at_sys_msuser (user_name,user_fullname, user_pass, user_email, created_by, created_at, modified_by,modified_at, is_active)
	VALUES (%s,'Admin Appsenin', %s,'rahmadhani.yusuf@iconpln.co.id','0',now(),'0',now(),TRUE);
	"""
	cur.execute(insert_query, (new_user['username'], new_user['password']))

	# Commit the transaction
	conn.commit()
except Exception as e:
	# Rollback the transaction in case of error
	conn.rollback()
	print(f"An error occurred: {e}")

In [20]:
try:
    # Fetch user data from the database
    cur.execute("SELECT user_pass FROM at_sys_msuser WHERE user_name = %s", (new_user['username'],))
    result = cur.fetchone()
    
    if result:
        stored_password = result[0]
        # Test the password
        is_valid = decode_password(stored_password, "secureAdmin#2025")
        print(f"Password is valid: {is_valid}")
    else:
        print("User not found.")
except Exception as e:
    print(f"An error occurred: {e}")

Password is valid: True
